In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [161]:
#res_file =  "res"
res_file =  "res_cnn"
#res_file = "res_ted-ru-en"
#res_file = "res_cnn_ted-ru-en"
#res_file = "res_robust19-en-ja"
#res_file = "res_robust19-ja-en"
with open(res_file) as f:
    res = f.readlines()
res = eval(res[0])

In [154]:
res[100]

({'architecture': 'rnn',
  'batch_size': 4096.0,
  'bpe_symbols': 10000.0,
  'initial_learning_rate': 0.0006,
  'num_embed': 256.0,
  'num_layers': 4.0,
  'rnn_cell_type': 'gru',
  'rnn_num_hidden': 512.0},
 {'dev_cpu_time': 661.677,
  'dev_ppl': 36.382934,
  'gpu_memory': 6671,
  'num_param': 24043796,
  'num_updates': 404000})

In [149]:
res[500]

({'architecture': 'trans',
  'batch_size': 4096.0,
  'bpe_symbols': 50000.0,
  'initial_learning_rate': 0.001,
  'num_embed': 256.0,
  'num_layers': 2.0,
  'transformer_attention_heads': 8.0,
  'transformer_feed_forward_num_hidden': 2048.0,
  'transformer_model_size': 256.0},
 {'dev_cpu_time': 429.392,
  'dev_ppl': 27.935406,
  'gpu_memory': 5089,
  'num_param': 27952724,
  'num_updates': 40000})

In [14]:
len(res)

414

In [156]:
#rnn_hyps = {'batch_size':[2048,4096],'initial_learning_rate':[0.0003, 0.0006, 0.001], 'num_embed':[512,1024], 'num_layers':[1,2], 'bpe_symbols':[10000,30000,50000],'rnn_cell_type':["lstm", "gru"], 'rnn_num_hidden':[512,1024]}
rnn_hyps = {'batch_size':[2048,4096],'initial_learning_rate':[0.0003, 0.0006, 0.001], 'num_embed':[256,512,1024], 'num_layers':[1,2,4], 'rnn_cell_type':["lstm", "gru"], 'rnn_num_hidden':[256,512,1024]}
#cnn_hyps = {'cnn_kernel_width':[3,5],'cnn_num_hidden':[256,1024],'initial_learning_rate':[0.0003, 0.0006, 0.001],'num_embed':[256,1024],'num_layers':[10, 20],'bpe_symbols':[10000,30000,50000]}
cnn_hyps = {'cnn_kernel_width':[3,5],'cnn_num_hidden':[256,1024],'initial_learning_rate':[0.0003, 0.0006, 0.001],'num_embed':[256,1024],'num_layers':[10, 20]}
#trans_hyps = {'initial_learning_rate':[0.0003, 0.0006, 0.001],'num_embed':[256, 512, 1024],'num_layers':[2,4], 'bpe_symbols':[10000,30000,50000],'transformer_attention_heads':[8,16],'transformer_feed_forward_num_hidden':[1024, 2048],'transformer_model_size':[256, 512,1024]}
trans_hyps = {'initial_learning_rate':[0.0003, 0.0006, 0.001],'num_embed':[256, 512, 1024],'num_layers':[2,4], 'transformer_attention_heads':[8,16],'transformer_feed_forward_num_hidden':[1024, 2048],'transformer_model_size':[256, 512,1024]}


In [123]:
def make_lst(oneres, hyp_lst):
    vals = [oneres[0][h] for h in hyp_lst]
    return vals
def get_single_eval(res, archi, hyp, eva, hyp_vals):
    if archi=="rnn":
        hyp_lst = list(rnn_hyps)
    elif archi=="cnn":
        hyp_lst = list(cnn_hyps)
    elif archi=="trans":
        hyp_lst = list(trans_hyps)
    hyp_lst.remove(hyp)
    vals_lst = []
    res_lst = []
    for i in range(len(res)):
        if res[i][0]["architecture"]==archi:
            vals_lst.append(make_lst(res[i],hyp_lst))
            res_lst.append(res[i])
    if len(hyp_vals)==2:
        hv = {hyp_vals[0]:[],hyp_vals[1]:[]}
        for i in range(len(vals_lst)-1):
            for j in range(i+1, len(vals_lst)):
                if cmp(vals_lst[i], vals_lst[j])==0:
                    if res_lst[i][0][hyp]==res_lst[j][0][hyp]:
                        continue
                    hv[res_lst[i][0][hyp]].append(res_lst[i][1][eva])
                    hv[res_lst[j][0][hyp]].append(res_lst[j][1][eva])
    elif len(hyp_vals)==3:
        hv = {hyp_vals[0]:[],hyp_vals[1]:[], hyp_vals[2]:[]}
        for i in range(len(vals_lst)-2):
            for j in range(i+1, len(vals_lst)-1):
                for m in range(i+2, len(vals_lst)):
                    if cmp(vals_lst[i], vals_lst[j])==0 and cmp(vals_lst[i], vals_lst[m])==0:
                        #print(res_lst[i][0], res_lst[j][0], res_lst[m][0])
                        if res_lst[i][0][hyp]==res_lst[j][0][hyp] or res_lst[m][0][hyp]==res_lst[j][0][hyp] or res_lst[i][0][hyp]==res_lst[m][0][hyp]:
                            continue
                        hv[res_lst[i][0][hyp]].append(res_lst[i][1][eva])
                        hv[res_lst[j][0][hyp]].append(res_lst[j][1][eva])
                        hv[res_lst[m][0][hyp]].append(res_lst[m][1][eva])
    return hv

In [128]:
def get_real_single_eval(res, archi, hyp, eva, hyp_vals):
    if len(hyp_vals)==2:
        hv = {hyp_vals[0]:[],hyp_vals[1]:[]}
    elif len(hyp_vals)==3:
        hv = {hyp_vals[0]:[],hyp_vals[1]:[], hyp_vals[2]:[]}
    for i in range(len(res)):
        if res[i][0]["architecture"]==archi:
            hv[res[i][0][hyp]].append(res[i][1][eva])
    return hv
        

In [50]:
def get_diff(hv):
    diff = {}
    keys = sorted(hv.keys())
    if len(keys)>=2:
        diff[str(keys[0])+"-"+str(keys[1])] = sorted(np.array(hv[keys[0]])-np.array(hv[keys[1]]))
        if len(keys)==3:
            diff[str(keys[0])+"-"+str(keys[2])] = sorted(np.array(hv[keys[1]])-np.array(hv[keys[2]]))
    return diff

In [122]:
#def draw_diff(hyp_dic, hyp, res, arch, eva):
def draw_diff(hv, arch, hyp):
    
#     if len(hyp_dic[hyp])>=2:
#         hv = get_single_eval(res, arch, hyp, eva, hyp_dic[hyp][:2])
#         if len(hyp_dic[hyp])==3:
#             hv.update(get_single_eval(res, arch, hyp, eva, [hyp_dic[hyp][i] for i in [0,-1]]))
    diff = get_diff(hv)
    x = np.arange(1, len(hv[hv.keys()[0]])+1)
    y = np.array(diff[diff.keys()[0]])
    y_ = None
    if len(diff) >1:
        y_ = np.array(diff[diff.keys()[1]])
    #plt.ylim(20,150)
    plt.clf()
    plt.plot(np.arange(1, len(hv[hv.keys()[0]])+1), np.zeros(len(hv[hv.keys()[0]])))
    plt.plot(x,y,'.',color="orange", label=hyp+":"+diff.keys()[0])
    if y_ is not None:
        plt.plot(x,y_,'.',color="red", label=hyp+":"+diff.keys()[1])
    plt.legend()
    #plt.show()
    plt.savefig(res_file+"_" + arch+ "_" + hyp+".png",format="png",frameon=False, bbox_inches = 'tight', pad_inches=0)

In [98]:
def draw_bar(hv, arch, hyp):
    keys = sorted(hv.keys())
    numk = len(keys)
    
    plt.clf()
    x = np.arange(1,numk+1)
    plt.xticks(np.arange(1,numk+1), keys)
    y = np.array([sum(hv[k])/len(hv[k]) for k in keys])
    e = np.array([np.std(hv[k]) for k in keys])
    plt.errorbar(x, y, e, linestyle='None', marker="o", ecolor="lightblue", elinewidth=3, capsize=5, label=hyp)
    plt.legend(loc=0)
    plt.savefig("err_" + res_file+"_" + arch+ "_" + hyp+".png",format="png",frameon=False, bbox_inches = 'tight', pad_inches=0)


In [162]:
arch = "cnn"
eva = "train_ppl"

if arch == "rnn":
    hyp_dic = rnn_hyps
elif arch == "cnn":
    hyp_dic = cnn_hyps
elif arch == "trans":
    hyp_dic = trans_hyps

for hyp in hyp_dic.keys():
    hv = get_single_eval(res, arch, hyp, eva, hyp_dic[hyp])
    if len(hv[hv.keys()[0]]) > 0:
        draw_diff(hv, arch, hyp)
    #draw_diff(hyp_dic, hyp, res, arch, eva)


In [163]:
arch = "cnn"
eva = "train_ppl"

if arch == "rnn":
    hyp_dic = rnn_hyps
elif arch == "cnn":
    hyp_dic = cnn_hyps
elif arch == "trans":
    hyp_dic = trans_hyps
    
for hyp in hyp_dic.keys():
#for hyp in ["transformer_model_size"]:
    hv = get_real_single_eval(res, arch, hyp, eva, hyp_dic[hyp])
    if len(hv[hv.keys()[0]]) > 0:
        draw_bar(hv, arch, hyp)

{256: [], 512: [], 1024: []}
